# Determinação do tamanho dos arquivos binários com base no número de variáveis e tempos de previsão avaliados

O cálculo do tamanho dos arquivos binários com a distribuição espacial das estatísticas do SCANTEC, é feito da seguinte forma:

$$\text{size} = nvars \times (nlons \times nlats) \times 4\text{bytes} + (nvars \times 4\text{bytes}) + (nlevs \times 4\text{bytes})) \times ntempos$$

onde,

* $nvars$: é o número de variáveis consideradas na avaliação (vide arquivo `scantec.vars`;
* $nlons$: é o número de pontos de longitudes, calculado de acordo com a resolução para a qual os campos serão interpolados (eg., $nlons = \frac{360}{\text{res}}$ e $nlats = \frac{180}{\text{res}}$);
* $nlevs$: é a quantidade de níveis verticais escritos para a variável avaliada (no caso do SCANTEC, sempre será $nlevs = 1$, pois no arquivo binário, os records das variáveis são escritos para cada nível de forma sequencial);
* $ntempos$: é o número de tempos de previsão avaliados (eg., se `Forecast Time Step: 24`, então serão escritos dois tempos de previsão, referentes às `00` (tempo da análise) e `24` (previsão de 24 horas), independente do valor de `Ending Time`.

**Observação:** como o SCANTEC escreve os arquivos binários com 1 nível para cada variável, então a conta simplificada é:

$$\text{size} = (nvars \times (nlons \times nlats) \times 4\text{bytes} + (nvars \times 8\text{bytes})) \times ntempos$$

**Exemplos:**

Com 22 variáveis e 16 tempos:

```
Starting Time: 2020060100
Ending Time:   2020060500
Analisys Time Step:  24
Forecast Time Step:  24
Forecast Total Time: 360
```

$$(22 \times (385 \times 171) \times 4 + (22 \times 4) + ) \times 16 = 92.698.496 \text{ bytes}$$

Com 22 variáveis e 16 tempos:

```
Starting Time: 2020060100
Ending Time:   2020060100
Analisys Time Step:  24
Forecast Time Step:  24
Forecast Total Time: 360
```

$$(22 \times (385 \times 171) \times 4 + (22 \times 8)) \times 16 = 92.698.496 \text{ bytes}$$

Com 22 variáveis e 2 tempos:

```
Starting Time: 2020060100
Ending Time:   2020060100
Analisys Time Step:  24
Forecast Time Step:  24
Forecast Total Time: 24
```

$$(22 \times (385 \times 171) \times 4 + (22 \times 8)) \times 2 = 11.587.312 \text{ bytes}$$

Com 22 variáveis e 2 tempos:

```
Starting Time: 2020060100
Ending Time:   2020060500
Analisys Time Step:  24
Forecast Time Step:  24
Forecast Total Time: 24
```

$$(22 \times (385 \times 171) \times 4 + (22 \times 8)) \times 2 = 11.587.312 \text{ bytes}$$

Com 20 variáveis e 4 tempos:

```
Starting Time: 2014080500 
Ending Time: 2014080600   
Analisys Time Step: 12 
Forecast Time Step: 24 
Forecast Total Time: 72
```

$$(20 \times (119 \times 154) \times 4 + (20 \times 8)) \times 4 = 5.864.960 \text{ bytes}$$

In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
#from IPython.display import display, Math, Latex
#display(Math(r'F(k) = \int_{-\infty}^{\infty} f(x) e^{2\pi i k} dx'))

In [ ]:
path = '/home/carlos/Downloads/SCANPLOT_T11212/test/SCANTEC.TESTS/dataout/'
mfile = 'MEANX126_20200601002020081500F.scan'

fname = os.path.join(path, mfile)

In [ ]:
fname

In [ ]:
xdef = 385
ydef = 171
zdef = 1
tdef = 8
nvars = 15

#lats = np.linspace(-90,-45, ydef)
lats = np.linspace(-90,90, ydef)
lons = np.linspace(0,360, xdef)

size = (nvars * (xdef * ydef) * 4 + (nvars * 8)) * tdef

times = np.arange(tdef)

In [ ]:
size

In [ ]:
dtype = np.dtype([
    ('psnm000', np.float32),
    ('temp850', np.float32),
    ('temp500', np.float32),
    ('temp250', np.float32),
    ('umes925', np.float32),
    ('agpl925', np.float32),
    ('zgeo850', np.float32),
    ('zgeo500', np.float32),
    ('zgeo250', np.float32),
    ('uvel850', np.float32),
    ('uvel500', np.float32),
    ('uvel250', np.float32),
    ('vvel850', np.float32),
    ('vvel500', np.float32),
    ('vvel250', np.float32),
])


f = open(fname, 'rb')
f.seek(60, os.SEEK_SET) # 60 = 15 variáveis * 4 bytes

data = np.fromfile(f, dtype=dtype)

In [ ]:
data

In [ ]:
data.shape

In [ ]:
da = xr.DataArray(data)

In [ ]:
da

In [ ]:
with open(fname,'rb') as f:
    for t in np.arange(tdef):
        for i in np.arange(nvars):
            f.seek(4, os.SEEK_SET)
            data = np.fromfile(f, dtype=np.float32, count=xdef*ydef)#, offset=8)
            field = np.reshape(data, (xdef, ydef), order='F')

In [ ]:
field.shape

In [ ]:
lista_n = []

if os.path.exists(fname):

    print(fname)

    dsl = []

    ds = xr.Dataset()

    with open(fname,'rb') as f:

        for t in np.arange(tdef):

            for i in np.arange(nvars):

                #s2d = str(xdef*ydef) + 'float32'

                #pad = '8b'  

                #dt = [ ('pad1', pad),  ('field', s2d), ('pad2', pad) ]    

                #dt_obj = np.dtype(dt)#, align=True)    

                #data = np.fromfile(f, dtype=np.float32, count=xdef*ydef, offset=8)    

                f.seek(4, os.SEEK_SET)
                
                data = np.fromfile(f, dtype=np.float32, count=xdef*ydef)#, offset=8)

                field = np.reshape(data, (xdef, ydef), order='F')

                field[field==-999.9]=np.nan
                
                
#                print(t, i)
#
#                ds[fnames[i]] = (('lon','lat'), field)
                ds[fname] = (('lon','lat'), field)
                ds.coords['lat'] = ('lat', lats)
                ds.coords['lon'] = ('lon', lons)
                ds.coords['time'] = [times[t]]
                dst = ds.transpose('time', 'lat', 'lon')
            dsl.append(dst)
        dsc = xr.concat(dsl, dim='time')
#        ds_field[ntpath.basename(str(fname))] = xr.concat(dsl, dim='time')

In [ ]:
dsc

In [ ]:
ds = dsc.isel(time=0).to_array(dim='test')

In [ ]:
ds.min(), ds.max()

In [ ]:
ds.plot()

In [ ]:
#5865088

In [ ]:
#5864512

In [ ]:
#5864320

In [ ]:
#5864320

In [ ]:
#5864320 

In [ ]:
s2d = (np.float32,(tdef,xdef,ydef))
#s3d = str(ydef*xdef) + 'f4'
pad = np.int32 
#pad = 'i4' 

dt = [  
    ('p000', pad), #('p111', pad), ('p222', pad), ('p333', pad), 
    
    ('p1', pad),  ('vtmp925', s2d), ('p21', pad),
    ('p2', pad),  ('vtmp850', s2d), ('p22', pad),
    ('p3', pad),  ('vtmp500', s2d), ('p23', pad),       
    
    ('p444', pad), #('p555', pad), ('p666', pad), ('p777', pad), 
    
    ('p4', pad),  ('temp850', s2d), ('p24', pad),
    ('p5', pad),  ('temp500', s2d), ('p25', pad),
    ('p6', pad),  ('temp250', s2d), ('p26', pad),
    
    ('p888', pad), #('p999', pad), ('pa', pad), ('pb', pad),
    
    ('p7', pad),  ('psnm000', s2d), ('p27', pad),
    
    ('pc', pad), #('pd', pad), ('pe', pad), ('pf', pad), 
    
    ('p8', pad),  ('umes925', s2d), ('p28', pad),
    ('p9', pad),  ('umes850', s2d), ('p29', pad),
    ('p10', pad), ('umes500', s2d), ('p30', pad),
    
    ('pg', pad), #('ph', pad), ('pi', pad), ('pj', pad), 
    
    ('p11', pad), ('agpl925', s2d), ('p31', pad),
    
    ('pk', pad), #('pl', pad), ('pm', pad), ('pn', pad), 
    
    ('p12', pad), ('zgeo850', s2d), ('p32', pad),
    ('p13', pad), ('zgeo500', s2d), ('p33', pad),
    ('p14', pad), ('zgeo250', s2d), ('p34', pad),
    
    ('po', pad), #('pp', pad), ('pq', pad), ('pr', pad), 
    
    ('p15', pad), ('uvel850', s2d), ('p35', pad),
    ('p16', pad), ('uvel500', s2d), ('p36', pad),
    ('p17', pad), ('uvel250', s2d), ('p37', pad),
    
    ('ps', pad), #('pt', pad), ('pu', pad), ('pv', pad), 
    
    ('p18', pad), ('vvel850', s2d), ('p38', pad),
    ('p19', pad), ('vvel500', s2d), ('p39', pad),
    ('p20', pad), ('vvel250', s2d), ('p40', pad)    
]   

dt_obj = np.dtype(dt, align=True)

In [ ]:
#s2d = (np.float32, (nvars, tdef, xdef, ydef))
s2d = (np.float32, (xdef, ydef))

dt = [
      ('vtmp925t1', s2d), 
      ('vtmp850t1', s2d), 
      ('vtmp500t1', s2d),         
      ('temp850t1', s2d),
      ('temp500t1', s2d), 
      ('temp250t1', s2d),
      ('psnm000t1', s2d), 
      ('umes925t1', s2d), 
      ('umes850t1', s2d), 
      ('umes500t1', s2d), 
      ('agpl925t1', s2d), 
      ('zgeo850t1', s2d), 
      ('zgeo500t1', s2d),
      ('zgeo250t1', s2d), 
      ('uvel850t1', s2d), 
      ('uvel500t1', s2d), 
      ('uvel250t1', s2d), 
      ('vvel850t1', s2d), 
      ('vvel500t1', s2d), 
      ('vvel250t1', s2d), 
    
      ('vtmp925t2', s2d), 
      ('vtmp850t2', s2d), 
      ('vtmp500t2', s2d),         
      ('temp850t2', s2d),
      ('temp500t2', s2d), 
      ('temp250t2', s2d),
      ('psnm000t2', s2d), 
      ('umes925t2', s2d), 
      ('umes850t2', s2d), 
      ('umes500t2', s2d), 
      ('agpl925t2', s2d), 
      ('zgeo850t2', s2d), 
      ('zgeo500t2', s2d),
      ('zgeo250t2', s2d), 
      ('uvel850t2', s2d), 
      ('uvel500t2', s2d), 
      ('uvel250t2', s2d), 
      ('vvel850t2', s2d), 
      ('vvel500t2', s2d), 
      ('vvel250t2', s2d), 
    
      ('vtmp925t3', s2d), 
      ('vtmp850t3', s2d), 
      ('vtmp500t3', s2d),         
      ('temp850t3', s2d),
      ('temp500t3', s2d), 
      ('temp250t3', s2d),
      ('psnm000t3', s2d), 
      ('umes925t3', s2d), 
      ('umes850t3', s2d), 
      ('umes500t3', s2d), 
      ('agpl925t3', s2d), 
      ('zgeo850t3', s2d), 
      ('zgeo500t3', s2d),
      ('zgeo250t3', s2d), 
      ('uvel850t3', s2d), 
      ('uvel500t3', s2d), 
      ('uvel250t3', s2d), 
      ('vvel850t3', s2d), 
      ('vvel500t3', s2d), 
      ('vvel250t3', s2d), 
    
      ('vtmp925t4', s2d), 
      ('vtmp850t4', s2d), 
      ('vtmp500t4', s2d),         
      ('temp850t4', s2d),
      ('temp500t4', s2d), 
      ('temp250t4', s2d),
      ('psnm000t4', s2d), 
      ('umes925t4', s2d), 
      ('umes850t4', s2d), 
      ('umes500t4', s2d), 
      ('agpl925t4', s2d), 
      ('zgeo850t4', s2d), 
      ('zgeo500t4', s2d),
      ('zgeo250t4', s2d), 
      ('uvel850t4', s2d), 
      ('uvel500t4', s2d), 
      ('uvel250t4', s2d), 
      ('vvel850t4', s2d), 
      ('vvel500t4', s2d), 
      ('vvel250t4', s2d),     
     ]   

dt_obj = np.dtype(dt, align=True)

In [ ]:
f = open(file, 'rb')
#f.seek(1863, os.SEEK_SET)

data = np.fromfile(f, dtype=dt_obj, count=-1, offset=8)

In [ ]:
datar = np.reshape(data[0]['vvel250t4'], (xdef, ydef), order='F')

In [ ]:
xobj = xr.DataArray(datar)

In [ ]:
xobj.plot()

In [ ]:
s2d = (np.float32, (nvars, tdef, xdef, ydef))
pad = np.int32 

#dt = [ ('pad0', pad), ('var', s2d), ('pad1', pad) ] 
dt = [ ('var', s2d) ] 

dt_obj = np.dtype(dt, align=True)

In [ ]:
input_file = open(file,'rb')

data = np.fromfile(input_file, dtype=dt_obj, count=-1)#, offset=320)

In [ ]:
data

In [ ]:
datar = np.reshape(data[0]['var'], (nvars, tdef, xdef, ydef), order='F')

In [ ]:
datar

In [ ]:
xobj = xr.DataArray(datar)

In [ ]:
xobj.isel(dim_0=0, dim_1=0)

In [ ]:
xobj.isel(dim_0=0, dim_1=0).min(), xobj.isel(dim_0=0, dim_1=0).max()

In [ ]:
xobj.isel(dim_0=-1, dim_1=0).plot()